# Reliability: Timeout Management

----

This notebook focuses on **timeout strategies** for LLM applications.

You will learn:

- **TTFT vs Full Response**: Different timeout types for different needs
- **Streaming Timeouts**: Handle long-running streaming responses
- **Adaptive Timeouts**: Adjust based on request complexity
- **Timeout Recovery**: Handle timeouts gracefully

## Table of Contents

- [Understanding LLM Timeouts](#understanding-llm-timeouts)
- [Setup](#setup)
- [Part 1: TTFT vs Full Response Timeout](#part-1-ttft-vs-full-response-timeout)
- [Part 2: Streaming Timeout Management](#part-2-streaming-timeout-management)
- [Part 3: Adaptive Timeout Strategies](#part-3-adaptive-timeout-strategies)
- [Part 4: Timeout Recovery](#part-4-timeout-recovery)
- [Part 5: Production Timeout Manager](#part-5-production-timeout-manager)
- [Best Practices Summary](#best-practices-summary)
- [Wrap-up](#wrap-up)

## Understanding LLM Timeouts

### Types of Timeouts

```
Request Timeline:
─────────────────────────────────────────────────────────────────────
│ Connection │     TTFT     │            Token Generation            │
│  Timeout   │   Timeout    │              (Streaming)               │
├────────────┼──────────────┼────────────────────────────────────────┤
│   ~2s      │    ~10s      │           ~30-120s (varies)            │
─────────────────────────────────────────────────────────────────────
        ▲              ▲                    ▲
     Connect       First Token         Last Token
```

| Timeout Type | Description | Typical Value |
|--------------|-------------|---------------|
| **Connection** | Time to establish connection | 2-5s |
| **TTFT** | Time to First Token | 5-30s |
| **Read/Chunk** | Time between tokens | 10-30s |
| **Total** | End-to-end timeout | 60-300s |

### Why TTFT Matters

- **User Experience**: Users expect quick feedback
- **Early Failure Detection**: Know quickly if request is stuck
- **Resource Efficiency**: Don't wait forever for failed requests

## Setup

This notebook reuses the configuration file (`.foundry_config.json`) created by `0_setup/1_setup.ipynb`.

- If the file is missing, run the setup notebook first.
- Make sure you can authenticate (e.g., `az login`), so `DefaultAzureCredential` can work.

In [1]:
# Environment setup and PATH configuration
import json
import os
import subprocess
import time
import asyncio
import threading
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional, Callable, Generator
from dataclasses import dataclass, field
from concurrent.futures import ThreadPoolExecutor, TimeoutError as FuturesTimeoutError
from dotenv import load_dotenv

load_dotenv(override=True)

# Ensure the notebook kernel can find Azure CLI (`az`) on PATH
possible_paths = [
    '/opt/homebrew/bin',   # macOS (Apple Silicon)
    '/usr/local/bin',      # macOS (Intel) / Linux
    '/usr/bin',            # Linux / Codespaces
    '/home/linuxbrew/.linuxbrew/bin',  # Linux Homebrew
]

az_path = None
try:
    result = subprocess.run(['which', 'az'], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
        print(f'🔍 Azure CLI found: {result.stdout.strip()}')
except Exception:
    pass

paths_to_add: list[str] = []
if az_path and az_path not in os.environ.get('PATH', ''):
    paths_to_add.append(az_path)
else:
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get('PATH', ''):
            paths_to_add.append(path)

if paths_to_add:
    os.environ['PATH'] = ':'.join(paths_to_add) + ':' + os.environ.get('PATH', '')
    print(f"✅ Added to PATH: {', '.join(paths_to_add)}")
else:
    print('✅ PATH looks good already')

print(f"\nPATH (first 150 chars): {os.environ['PATH'][:150]}...")

🔍 Azure CLI found: /anaconda/envs/azureml_py38/bin//az
✅ PATH looks good already

PATH (first 150 chars): /anaconda/envs/azureml_py38/bin/:/afh/code/agent-operator-lab/.venv/bin:/home/azureuser/.vscode-server/cli/servers/Stable-c9d77990917f3102ada88be140d2...


In [2]:
# Load Foundry project settings from .foundry_config.json
from azure.identity import DefaultAzureCredential

config_file = '../0_setup/.foundry_config.json'
try:
    with open(config_file, 'r', encoding='utf-8') as f:
        config = json.load(f)
except FileNotFoundError as e:
    print(f"⚠️ Could not find '{config_file}'.")
    print('💡 Run 0_setup/1_setup.ipynb first to create it.')
    raise e

# Project variables from config
FOUNDRY_NAME = config.get('FOUNDRY_NAME')
RESOURCE_GROUP = config.get('RESOURCE_GROUP')
LOCATION = config.get('LOCATION')
AZURE_AI_PROJECT_ENDPOINT = config.get('AZURE_AI_PROJECT_ENDPOINT')

# Azure OpenAI variables from env
AZURE_OPENAI_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.environ.get("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME = os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.environ.get("AZURE_OPENAI_API_VERSION")

os.environ['FOUNDRY_NAME'] = FOUNDRY_NAME or ''
os.environ['LOCATION'] = LOCATION or ''
os.environ['RESOURCE_GROUP'] = RESOURCE_GROUP or ''
os.environ['AZURE_SUBSCRIPTION_ID'] = config.get('AZURE_SUBSCRIPTION_ID', '')

print(f"✅ Loaded settings from '{config_file}'.")
print(f"\n📌 Foundry name: {FOUNDRY_NAME}")
print(f"📌 Resource group: {RESOURCE_GROUP}")
print(f"📌 Location: {LOCATION}")
print(f"📌 Azure OpenAI endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"📌 Chat deployment: {AZURE_OPENAI_CHAT_DEPLOYMENT_NAME}")

# Initialize credential for Azure services
credential = DefaultAzureCredential()

✅ Loaded settings from '../0_setup/.foundry_config.json'.

📌 Foundry name: foundry-rq90gs
📌 Resource group: foundry-rg
📌 Location: swedencentral
📌 Azure OpenAI endpoint: https://foundry-rq90gs.cognitiveservices.azure.com/
📌 Chat deployment: gpt-4.1


## Part 1: TTFT vs Full Response Timeout with TimeoutInjector

Understand and implement different timeout types.

In [11]:
# TTFT vs Total Response Timeout
from openai import AzureOpenAI
import httpx
import threading

@dataclass
class TimeoutConfig:
    """Configuration for different timeout types."""
    connect_timeout: float = 5.0    # Time to establish connection
    ttft_timeout: float = 30.0      # Time to first token
    read_timeout: float = 60.0      # Time between chunks
    total_timeout: float = 300.0    # Total request timeout

@dataclass
class TimingResult:
    """Timing information for a request."""
    connect_time: Optional[float] = None
    ttft: Optional[float] = None
    total_time: Optional[float] = None
    token_count: int = 0
    timed_out: bool = False
    timeout_phase: Optional[str] = None
    response: Optional[str] = None

def create_client_with_timeouts(config: TimeoutConfig) -> AzureOpenAI:
    """Create Azure OpenAI client with custom timeouts."""
    http_client = httpx.Client(
        timeout=httpx.Timeout(
            connect=config.connect_timeout,
            read=config.read_timeout,
            write=30.0,
            pool=None,
        )
    )
    return AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
        http_client=http_client,
    )


# ── Timeout Injector ─────────────────────────────────────────
# Simulates delayed responses to trigger timeout scenarios.

class TimeoutInjector:
    """
    Wraps an API call and injects artificial delays
    to simulate TTFT, chunk, or total timeouts.
    """

    def __init__(
        self,
        real_func: Callable,
        delay_before_first: float = 0.0,   # Delay before TTFT
        delay_between_chunks: float = 0.0, # Delay between chunks
        fail_after_chunks: int = 0,        # Hang after N chunks
    ):
        self.real_func = real_func
        self.delay_before_first = delay_before_first
        self.delay_between_chunks = delay_between_chunks
        self.fail_after_chunks = fail_after_chunks

    def __call__(self, **kwargs):
        """
        Call the real function but inject delays.
        For streaming, wraps the generator.
        """
        if kwargs.get("stream"):
            return self._wrap_stream(**kwargs)
        else:
            # Non-streaming: just delay before response
            if self.delay_before_first > 0:
                print(
                    f"      💤 Injecting {self.delay_before_first}s "
                    f"delay before response..."
                )
                time.sleep(self.delay_before_first)
            return self.real_func(**kwargs)

    def _wrap_stream(self, **kwargs):
        """Wrap streaming response with delays."""
        stream = self.real_func(**kwargs)
        chunk_count = 0
        first_chunk = True

        for chunk in stream:
            # Delay before first token (TTFT simulation)
            if first_chunk and self.delay_before_first > 0:
                print(
                    f"      💤 Injecting {self.delay_before_first}s "
                    f"TTFT delay..."
                )
                time.sleep(self.delay_before_first)
                first_chunk = False

            # Hang after N chunks (simulates stalled connection)
            if (
                self.fail_after_chunks > 0
                and chunk_count >= self.fail_after_chunks
            ):
                print(
                    f"      🔥 Hanging after {chunk_count} chunks "
                    f"(simulating stall)..."
                )
                # Sleep longer than any reasonable timeout
                time.sleep(9999)

            # Delay between chunks
            if (
                not first_chunk
                and self.delay_between_chunks > 0
            ):
                time.sleep(self.delay_between_chunks)

            if chunk.choices and chunk.choices[0].delta.content:
                chunk_count += 1

            yield chunk


def measure_ttft_streaming(
    client: AzureOpenAI,
    messages: List[Dict[str, str]],
    deployment: str,
    config: TimeoutConfig,
    injector: Optional[TimeoutInjector] = None,
    **kwargs,
) -> TimingResult:
    """Measure TTFT using streaming response."""
    result = TimingResult()
    start_time = time.time()
    content_chunks = []

    try:
        # Use injector if provided, else call client directly
        if injector:
            stream = injector(
                model=deployment,
                messages=messages,
                stream=True,
                **kwargs,
            )
        else:
            stream = client.chat.completions.create(
                model=deployment,
                messages=messages,
                stream=True,
                **kwargs,
            )

        result.connect_time = time.time() - start_time

        for chunk in stream:
            current_time = time.time() - start_time

            # Check total timeout
            if current_time > config.total_timeout:
                result.timed_out = True
                result.timeout_phase = "total"
                break

            if chunk.choices and chunk.choices[0].delta.content:
                content = chunk.choices[0].delta.content
                content_chunks.append(content)
                result.token_count += 1

                # Record TTFT on first content
                if result.ttft is None:
                    result.ttft = current_time
                    # Check TTFT timeout
                    if result.ttft > config.ttft_timeout:
                        result.timed_out = True
                        result.timeout_phase = "ttft"
                        break

        result.total_time = time.time() - start_time
        result.response = ''.join(content_chunks)

    except httpx.TimeoutException as e:
        result.timed_out = True
        result.total_time = time.time() - start_time
        if "connect" in str(e).lower():
            result.timeout_phase = "connect"
        else:
            result.timeout_phase = "read"
    except Exception as e:
        result.total_time = time.time() - start_time
        result.response = f"Error: {str(e)}"

    return result


# ── Demo 1: Normal request (baseline) ────────────────────────
print("📊 TTFT Measurement Demo")
print("=" * 60)

if AZURE_OPENAI_ENDPOINT and AZURE_OPENAI_API_KEY:
    timeout_config = TimeoutConfig(
        connect_timeout=5.0,
        ttft_timeout=3.0,   # Tight TTFT for demo
        read_timeout=10.0,
        total_timeout=15.0,
    )

    print(f"\n⚙️ Timeout Configuration:")
    print(f"   Connect: {timeout_config.connect_timeout}s")
    print(f"   TTFT:    {timeout_config.ttft_timeout}s")
    print(f"   Read:    {timeout_config.read_timeout}s")
    print(f"   Total:   {timeout_config.total_timeout}s")

    client = create_client_with_timeouts(timeout_config)
    messages = [
        {"role": "user", "content": "Say 'hello' only."}
    ]

    # ── Baseline: no injection ──
    print(f"\n{'─'*60}")
    print("🟢 Test 1: Normal request (no injection)")
    result = measure_ttft_streaming(
        client=client,
        messages=messages,
        deployment=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
        config=timeout_config,
        max_tokens=10,
    )
    print(f"   TTFT: {result.ttft*1000:.0f}ms" if result.ttft else "   TTFT: N/A")
    print(f"   Total: {result.total_time*1000:.0f}ms")
    print(f"   Timed out: {result.timed_out}")
    print(f"   Response: {result.response}")

    # ── Inject TTFT delay → should trigger ttft timeout ──
    print(f"\n{'─'*60}")
    print("🔥 Test 2: Injecting 4s TTFT delay (timeout=3s)")

    def real_create(**kw):
        return client.chat.completions.create(**kw)

    injector_ttft = TimeoutInjector(
        real_create,
        delay_before_first=4.0,  # > ttft_timeout
    )

    result = measure_ttft_streaming(
        client=client,
        messages=messages,
        deployment=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
        config=timeout_config,
        injector=injector_ttft,
        max_tokens=10,
    )
    print(f"   TTFT: {result.ttft*1000:.0f}ms" if result.ttft else "   TTFT: N/A")
    print(f"   Total: {result.total_time*1000:.0f}ms")
    print(f"   Timed out: {result.timed_out}")
    print(f"   Timeout phase: {result.timeout_phase}")
    print(f"   Response: {result.response or '(none)'}")

    # ── Inject chunk stall → total timeout ──
    print(f"\n{'─'*60}")
    print("🔥 Test 3: Stall after 2 chunks (total_timeout=15s)")

    # Use a thread with a hard kill for the stall test
    # because the injector sleeps forever
    def run_stall_test():
        injector_stall = TimeoutInjector(
            real_create,
            fail_after_chunks=2,
        )
        return measure_ttft_streaming(
            client=client,
            messages=[
                {"role": "user", "content": "Count 1 2 3 4 5"}
            ],
            deployment=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
            config=TimeoutConfig(
                ttft_timeout=5.0,
                read_timeout=2.0,  # Will trigger read timeout
                total_timeout=5.0,
            ),
            injector=injector_stall,
            max_tokens=50,
        )

    # Run in a thread with a timeout
    result_container = [None]

    def target():
        result_container[0] = run_stall_test()

    t = threading.Thread(target=target, daemon=True)
    t.start()
    t.join(timeout=6.0)

    if result_container[0]:
        result = result_container[0]
        print(f"   Tokens received: {result.token_count}")
        print(f"   Timed out: {result.timed_out}")
        print(f"   Timeout phase: {result.timeout_phase}")
        print(f"   Partial response: {result.response or '(none)'}")
    else:
        print("   ⚠️ Test killed by thread timeout (as expected)")
        print("   ✅ Demonstrates that without proper read "
              "timeout, request would hang forever")

else:
    print(
        "⚠️ AZURE_OPENAI_ENDPOINT or "
        "AZURE_OPENAI_API_KEY not set"
    )

📊 TTFT Measurement Demo

⚙️ Timeout Configuration:
   Connect: 5.0s
   TTFT:    3.0s
   Read:    10.0s
   Total:   15.0s

────────────────────────────────────────────────────────────
🟢 Test 1: Normal request (no injection)


   TTFT: 763ms
   Total: 764ms
   Timed out: False
   Response: Hello

────────────────────────────────────────────────────────────
🔥 Test 2: Injecting 4s TTFT delay (timeout=3s)
      💤 Injecting 4.0s TTFT delay...
   TTFT: 4532ms
   Total: 4532ms
   Timed out: True
   Timeout phase: ttft
   Response: Hello

────────────────────────────────────────────────────────────
🔥 Test 3: Stall after 2 chunks (total_timeout=15s)
      🔥 Hanging after 2 chunks (simulating stall)...
   ⚠️ Test killed by thread timeout (as expected)
   ✅ Demonstrates that without proper read timeout, request would hang forever


## Part 2: Streaming Timeout Management

Handle timeouts in streaming scenarios.

In [12]:
# Streaming Timeout Handler

@dataclass
class StreamingConfig:
    """Configuration for streaming timeout handling."""
    chunk_timeout: float = 10.0    # Max time between chunks
    total_timeout: float = 120.0   # Total streaming timeout
    min_chunks: int = 1            # Minimum chunks before timeout OK

class StreamingTimeoutHandler:
    """Handle timeouts during streaming responses."""

    def __init__(self, config: StreamingConfig):
        self.config = config
        self.start_time: Optional[float] = None
        self.last_chunk_time: Optional[float] = None
        self.chunk_count: int = 0

    def start(self) -> None:
        """Start timing."""
        self.start_time = time.time()
        self.last_chunk_time = self.start_time
        self.chunk_count = 0

    def record_chunk(self) -> None:
        """Record a chunk received."""
        self.last_chunk_time = time.time()
        self.chunk_count += 1

    def check_timeout(self) -> tuple[bool, str]:
        """Check if timeout occurred."""
        if self.start_time is None:
            return False, ""

        current_time = time.time()

        # Check total timeout
        elapsed_total = current_time - self.start_time
        if elapsed_total > self.config.total_timeout:
            return True, (
                f"total_timeout ({elapsed_total:.1f}s "
                f"> {self.config.total_timeout}s)"
            )

        # Check chunk timeout (only after min_chunks)
        if (
            self.chunk_count >= self.config.min_chunks
            and self.last_chunk_time
        ):
            elapsed_since_chunk = current_time - self.last_chunk_time
            if elapsed_since_chunk > self.config.chunk_timeout:
                return True, (
                    f"chunk_timeout ({elapsed_since_chunk:.1f}s "
                    f"> {self.config.chunk_timeout}s)"
                )

        return False, ""

    def get_stats(self) -> Dict[str, Any]:
        """Get timing statistics."""
        current_time = time.time()
        elapsed = (
            current_time - self.start_time
            if self.start_time else 0
        )
        return {
            "elapsed_total": elapsed,
            "time_since_last_chunk": (
                current_time - self.last_chunk_time
                if self.last_chunk_time else 0
            ),
            "chunk_count": self.chunk_count,
            "chunks_per_second": (
                self.chunk_count / elapsed
                if elapsed > 0 else 0
            ),
        }


# ── Streaming with injected delays ────────────────────────
class StreamingChunkInjector:
    """
    Wraps a streaming response to inject delays
    between chunks for timeout testing.
    """

    def __init__(
        self,
        real_create_func: Callable,
        delay_after_n_chunks: int = 0,
        delay_seconds: float = 0.0,
    ):
        self.real_create_func = real_create_func
        self.delay_after_n_chunks = delay_after_n_chunks
        self.delay_seconds = delay_seconds

    def create(self, **kwargs):
        """Create streaming response with injected delay."""
        stream = self.real_create_func(**kwargs)
        chunk_count = 0

        for chunk in stream:
            if chunk.choices and chunk.choices[0].delta.content:
                chunk_count += 1
                # Inject delay after N chunks
                if (
                    self.delay_after_n_chunks > 0
                    and chunk_count == self.delay_after_n_chunks
                ):
                    print(
                        f"\n      💤 Injecting {self.delay_seconds}s "
                        f"delay after chunk {chunk_count}..."
                    )
                    time.sleep(self.delay_seconds)
            yield chunk


def stream_with_timeout_v2(
    stream_source,
    streaming_config: StreamingConfig,
) -> Generator[str, None, Dict[str, Any]]:
    """
    Stream response with timeout handling.
    Accepts any iterable of chunks.
    """
    handler = StreamingTimeoutHandler(streaming_config)
    handler.start()

    for chunk in stream_source:
        timed_out, reason = handler.check_timeout()
        if timed_out:
            print(f"\n   ⚠️ Timeout: {reason}")
            break

        if chunk.choices and chunk.choices[0].delta.content:
            content = chunk.choices[0].delta.content
            handler.record_chunk()
            yield content

    return handler.get_stats()


# ── Demo ─────────────────────────────────────────────────────
print("📊 Streaming Timeout Demo")
print("=" * 60)

if AZURE_OPENAI_ENDPOINT and AZURE_OPENAI_API_KEY:
    client = AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
    )

    # ── Test 1: Normal streaming (baseline) ──
    print("\n🟢 Test 1: Normal streaming (no injection)")
    streaming_config = StreamingConfig(
        chunk_timeout=5.0,
        total_timeout=30.0,
        min_chunks=1,
    )

    print(f"   Config: chunk_timeout={streaming_config.chunk_timeout}s, "
          f"total_timeout={streaming_config.total_timeout}s")

    messages = [
        {"role": "user", "content": "Say 'hello world' only."}
    ]

    print("   Response: ", end="", flush=True)
    content_chunks = []
    stream = client.chat.completions.create(
        model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
        messages=messages,
        stream=True,
        max_tokens=20,
    )
    for chunk in stream_with_timeout_v2(stream, streaming_config):
        print(chunk, end="", flush=True)
        content_chunks.append(chunk)
    print(f"\n   ✅ Completed ({len(content_chunks)} chunks)")

    # ── Test 2: Inject chunk delay → trigger chunk_timeout ──
    print(f"\n{'─'*60}")
    print("🔥 Test 2: Inject 6s delay after chunk 2 "
          "(chunk_timeout=3s)")

    tight_config = StreamingConfig(
        chunk_timeout=3.0,   # Tight timeout
        total_timeout=30.0,
        min_chunks=1,
    )

    def real_create(**kw):
        return client.chat.completions.create(**kw)

    injector = StreamingChunkInjector(
        real_create,
        delay_after_n_chunks=2,
        delay_seconds=6.0,  # > chunk_timeout
    )

    messages = [
        {"role": "user", "content": "Count from 1 to 10"}
    ]

    print("   Response: ", end="", flush=True)
    content_chunks = []
    stream = injector.create(
        model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
        messages=messages,
        stream=True,
        max_tokens=50,
    )
    for chunk in stream_with_timeout_v2(stream, tight_config):
        print(chunk, end="", flush=True)
        content_chunks.append(chunk)

    print(f"\n   Partial response: {len(content_chunks)} chunks")
    print("   ✅ Chunk timeout correctly detected and handled")

    # ── Test 3: Inject delay → trigger total_timeout ──
    print(f"\n{'─'*60}")
    print("🔥 Test 3: Inject 8s delay "
          "(total_timeout=5s)")

    very_tight_config = StreamingConfig(
        chunk_timeout=10.0,
        total_timeout=5.0,  # Will be exceeded
        min_chunks=1,
    )

    injector2 = StreamingChunkInjector(
        real_create,
        delay_after_n_chunks=1,
        delay_seconds=8.0,  # > total_timeout
    )

    print("   Response: ", end="", flush=True)
    content_chunks = []
    stream = injector2.create(
        model=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
        messages=[{"role": "user", "content": "Say 'A B C'"}],
        stream=True,
        max_tokens=20,
    )
    for chunk in stream_with_timeout_v2(stream, very_tight_config):
        print(chunk, end="", flush=True)
        content_chunks.append(chunk)

    print(f"\n   Partial response: {len(content_chunks)} chunks")
    print("   ✅ Total timeout correctly detected")

else:
    print(
        "⚠️ AZURE_OPENAI_ENDPOINT or "
        "AZURE_OPENAI_API_KEY not set"
    )

📊 Streaming Timeout Demo

🟢 Test 1: Normal streaming (no injection)
   Config: chunk_timeout=5.0s, total_timeout=30.0s
   Response: 

hello world
   ✅ Completed (2 chunks)

────────────────────────────────────────────────────────────
🔥 Test 2: Inject 6s delay after chunk 2 (chunk_timeout=3s)
   Response: Sure
      💤 Injecting 6.0s delay after chunk 2...

   ⚠️ Timeout: chunk_timeout (6.0s > 3.0s)

   Partial response: 1 chunks
   ✅ Chunk timeout correctly detected and handled

────────────────────────────────────────────────────────────
🔥 Test 3: Inject 8s delay (total_timeout=5s)
   Response: 
      💤 Injecting 8.0s delay after chunk 1...

   ⚠️ Timeout: total_timeout (8.7s > 5.0s)

   Partial response: 0 chunks
   ✅ Total timeout correctly detected


## Part 3: Adaptive Timeout Strategies

Adjust timeouts based on request characteristics.

In [13]:
# Adaptive Timeout Calculator

@dataclass
class RequestCharacteristics:
    """Characteristics that affect timeout."""
    estimated_input_tokens: int
    max_output_tokens: int
    model: str
    is_complex_task: bool = False  # e.g., code generation, analysis
    has_tools: bool = False

class AdaptiveTimeoutCalculator:
    """
    Calculate appropriate timeouts based on request.
    """
    
    # Approximate tokens per second for different models
    MODEL_SPEEDS = {
        "gpt-4o": 80,         # tokens/second
        "gpt-4o-mini": 120,
        "gpt-4.1": 70,
        "gpt-4": 40,
        "gpt-35-turbo": 100,
        "o1": 30,             # Reasoning models are slower
        "o3": 25,
    }
    
    # Base TTFT for different models (seconds)
    BASE_TTFT = {
        "gpt-4o": 1.0,
        "gpt-4o-mini": 0.5,
        "gpt-4.1": 1.5,
        "gpt-4": 2.0,
        "gpt-35-turbo": 0.5,
        "o1": 5.0,            # Reasoning models have higher TTFT
        "o3": 10.0,
    }
    
    def __init__(
        self,
        min_timeout: float = 10.0,
        max_timeout: float = 300.0,
        safety_multiplier: float = 2.0,
    ):
        self.min_timeout = min_timeout
        self.max_timeout = max_timeout
        self.safety_multiplier = safety_multiplier
    
    def _get_model_base(self, model: str) -> str:
        """Get base model name for lookup."""
        model_lower = model.lower()
        for base in self.MODEL_SPEEDS.keys():
            if base in model_lower:
                return base
        return "gpt-4o"  # default
    
    def calculate_ttft_timeout(self, characteristics: RequestCharacteristics) -> float:
        """
        Calculate TTFT timeout based on request.
        """
        base_model = self._get_model_base(characteristics.model)
        base_ttft = self.BASE_TTFT.get(base_model, 2.0)
        
        # Adjust for input size (longer prompts = longer TTFT)
        input_adjustment = characteristics.estimated_input_tokens / 1000 * 0.5
        
        # Adjust for complexity
        complexity_multiplier = 1.5 if characteristics.is_complex_task else 1.0
        
        # Adjust for tools
        tools_adjustment = 2.0 if characteristics.has_tools else 0.0
        
        ttft = (base_ttft + input_adjustment + tools_adjustment) * complexity_multiplier
        ttft *= self.safety_multiplier
        
        return max(self.min_timeout / 3, min(ttft, self.max_timeout / 3))
    
    def calculate_total_timeout(self, characteristics: RequestCharacteristics) -> float:
        """
        Calculate total timeout based on request.
        """
        base_model = self._get_model_base(characteristics.model)
        tokens_per_second = self.MODEL_SPEEDS.get(base_model, 50)
        
        # Estimate generation time
        generation_time = characteristics.max_output_tokens / tokens_per_second
        
        # Add TTFT
        ttft = self.calculate_ttft_timeout(characteristics)
        
        # Total with safety margin
        total = (ttft + generation_time) * self.safety_multiplier
        
        return max(self.min_timeout, min(total, self.max_timeout))
    
    def calculate_timeouts(self, characteristics: RequestCharacteristics) -> TimeoutConfig:
        """
        Calculate complete timeout configuration.
        """
        ttft = self.calculate_ttft_timeout(characteristics)
        total = self.calculate_total_timeout(characteristics)
        
        return TimeoutConfig(
            connect_timeout=5.0,
            ttft_timeout=ttft,
            read_timeout=max(ttft, 30.0),
            total_timeout=total,
        )

# Demonstrate adaptive timeouts
print("📊 Adaptive Timeout Calculator Demo")
print("=" * 60)

calculator = AdaptiveTimeoutCalculator(
    min_timeout=10.0,
    max_timeout=300.0,
    safety_multiplier=2.0,
)

# Test different scenarios
scenarios = [
    ("Simple query (gpt-4o-mini)", RequestCharacteristics(
        estimated_input_tokens=100,
        max_output_tokens=100,
        model="gpt-4o-mini",
    )),
    ("Complex analysis (gpt-4o)", RequestCharacteristics(
        estimated_input_tokens=2000,
        max_output_tokens=2000,
        model="gpt-4o",
        is_complex_task=True,
    )),
    ("Code generation with tools (gpt-4.1)", RequestCharacteristics(
        estimated_input_tokens=1000,
        max_output_tokens=4000,
        model="gpt-4.1",
        is_complex_task=True,
        has_tools=True,
    )),
    ("Reasoning task (o1)", RequestCharacteristics(
        estimated_input_tokens=500,
        max_output_tokens=1000,
        model="o1",
        is_complex_task=True,
    )),
]

print("\n📋 Calculated Timeouts by Scenario:")
print("-" * 80)
print(f"{'Scenario':<35} {'TTFT':<10} {'Read':<10} {'Total':<10}")
print("-" * 80)

for name, characteristics in scenarios:
    config = calculator.calculate_timeouts(characteristics)
    print(f"{name:<35} {config.ttft_timeout:<10.1f} {config.read_timeout:<10.1f} {config.total_timeout:<10.1f}")

📊 Adaptive Timeout Calculator Demo

📋 Calculated Timeouts by Scenario:
--------------------------------------------------------------------------------
Scenario                            TTFT       Read       Total     
--------------------------------------------------------------------------------
Simple query (gpt-4o-mini)          3.3        30.0       10.0      
Complex analysis (gpt-4o)           6.0        30.0       62.0      
Code generation with tools (gpt-4.1) 12.0       30.0       138.3     
Reasoning task (o1)                 15.8       30.0       98.2      


## Part 4: Timeout Recovery

Handle timeouts gracefully with recovery strategies.

In [14]:
# Timeout Recovery Strategies

class RecoveryStrategy(Enum):
    """Strategies for handling timeouts."""
    RETRY_SAME = "retry_same"
    RETRY_SIMPLIFIED = "retry_simplified"
    RETRY_FASTER_MODEL = "retry_faster"
    FALLBACK_CACHE = "fallback_cache"
    PARTIAL_RESPONSE = "partial_response"
    FAIL_FAST = "fail_fast"

@dataclass
class RecoveryResult:
    """Result of recovery attempt."""
    success: bool
    strategy_used: RecoveryStrategy
    attempts: int
    response: Optional[str]
    partial: bool = False
    timed_out: bool = False


class TimeoutRecoveryManager:
    """Manage timeout recovery with multiple strategies."""

    MODEL_FALLBACKS = {
        "gpt-4o": "gpt-4o-mini",
        "gpt-4.1": "gpt-4o",
        "o1": "gpt-4o",
        "o3": "o1",
    }

    def __init__(
        self,
        client: AzureOpenAI,
        default_deployment: str,
        max_retries: int = 2,
    ):
        self.client = client
        self.default_deployment = default_deployment
        self.max_retries = max_retries
        # Injector for testing (set externally)
        self.timeout_injector: Optional[Callable] = None

    def _try_request(
        self,
        messages: List[Dict[str, str]],
        deployment: str,
        timeout_config: TimeoutConfig,
        max_tokens: int,
    ) -> tuple[bool, Optional[str], bool]:
        """
        Try a request with timeout.
        Returns (success, response, timed_out).
        """
        partial_content = []
        timed_out = False

        try:
            # Use injector if set, else real client
            if self.timeout_injector:
                stream = self.timeout_injector(
                    model=deployment,
                    messages=messages,
                    max_tokens=max_tokens,
                    stream=True,
                )
            else:
                stream = self.client.chat.completions.create(
                    model=deployment,
                    messages=messages,
                    max_tokens=max_tokens,
                    stream=True,
                )

            start_time = time.time()
            last_chunk_time = start_time

            for chunk in stream:
                current_time = time.time()

                # Check total timeout
                if (
                    current_time - start_time
                    > timeout_config.total_timeout
                ):
                    print(
                        f"      ⚠️ total_timeout exceeded "
                        f"({current_time - start_time:.1f}s)"
                    )
                    timed_out = True
                    break

                # Check read timeout
                if (
                    current_time - last_chunk_time
                    > timeout_config.read_timeout
                ):
                    print(
                        f"      ⚠️ read_timeout exceeded "
                        f"({current_time - last_chunk_time:.1f}s)"
                    )
                    timed_out = True
                    break

                if chunk.choices and chunk.choices[0].delta.content:
                    partial_content.append(
                        chunk.choices[0].delta.content
                    )
                    last_chunk_time = current_time

            response = ''.join(partial_content)
            success = len(response) > 0 and not timed_out

            return success, response, timed_out

        except Exception as e:
            response = (
                ''.join(partial_content)
                if partial_content else None
            )
            return False, response, True

    def complete_with_recovery(
        self,
        messages: List[Dict[str, str]],
        timeout_config: TimeoutConfig,
        max_tokens: int = 1000,
        strategies: List[RecoveryStrategy] = None,
    ) -> RecoveryResult:
        """Complete request with automatic recovery."""
        if strategies is None:
            strategies = [
                RecoveryStrategy.RETRY_SAME,
                RecoveryStrategy.RETRY_SIMPLIFIED,
                RecoveryStrategy.PARTIAL_RESPONSE,
            ]

        attempts = 0
        current_deployment = self.default_deployment
        current_max_tokens = max_tokens
        last_partial = None
        any_timeout = False

        for strategy in strategies:
            if attempts >= self.max_retries:
                break

            if strategy == RecoveryStrategy.RETRY_SIMPLIFIED:
                current_max_tokens = max_tokens // 2
                print(
                    f"      📉 Reducing max_tokens: "
                    f"{max_tokens} → {current_max_tokens}"
                )
            elif strategy == RecoveryStrategy.RETRY_FASTER_MODEL:
                fallback = self.MODEL_FALLBACKS.get(
                    current_deployment
                )
                if fallback:
                    print(
                        f"      🔄 Switching model: "
                        f"{current_deployment} → {fallback}"
                    )
                    current_deployment = fallback
            elif strategy == RecoveryStrategy.PARTIAL_RESPONSE:
                if last_partial:
                    return RecoveryResult(
                        success=True,
                        strategy_used=strategy,
                        attempts=attempts,
                        response=last_partial,
                        partial=True,
                        timed_out=any_timeout,
                    )
            elif strategy == RecoveryStrategy.FAIL_FAST:
                break

            attempts += 1
            print(
                f"   🔄 Attempt {attempts}: {strategy.value} "
                f"(max_tokens={current_max_tokens})"
            )

            success, response, timed_out = self._try_request(
                messages=messages,
                deployment=current_deployment,
                timeout_config=timeout_config,
                max_tokens=current_max_tokens,
            )

            if timed_out:
                any_timeout = True

            if response:
                last_partial = response

            if success:
                return RecoveryResult(
                    success=True,
                    strategy_used=strategy,
                    attempts=attempts,
                    response=response,
                    partial=False,
                    timed_out=False,
                )

        return RecoveryResult(
            success=False,
            strategy_used=RecoveryStrategy.FAIL_FAST,
            attempts=attempts,
            response=last_partial,
            partial=True if last_partial else False,
            timed_out=any_timeout,
        )


# ── Timeout Injector for Recovery Demo ────────────────────
class RecoveryTimeoutInjector:
    """
    Injects timeouts for the first N attempts,
    then allows success.
    """

    def __init__(
        self,
        real_create_func: Callable,
        fail_attempts: int = 2,
        delay_seconds: float = 5.0,
    ):
        self.real_create_func = real_create_func
        self.fail_attempts = fail_attempts
        self.delay_seconds = delay_seconds
        self.attempt_count = 0

    def __call__(self, **kwargs):
        """Wrap streaming with conditional delay."""
        self.attempt_count += 1
        stream = self.real_create_func(**kwargs)

        if self.attempt_count <= self.fail_attempts:
            # Inject delay on first N attempts
            return self._inject_delay(stream)
        else:
            # Let it succeed
            return stream

    def _inject_delay(self, stream):
        """Inject delay after first chunk."""
        chunk_count = 0
        for chunk in stream:
            if chunk.choices and chunk.choices[0].delta.content:
                chunk_count += 1
                if chunk_count == 1:
                    print(
                        f"      💤 Injecting {self.delay_seconds}s "
                        f"delay (attempt {self.attempt_count})..."
                    )
                    time.sleep(self.delay_seconds)
            yield chunk


# ── Demo ─────────────────────────────────────────────────────
print("📊 Timeout Recovery Demo")
print("=" * 60)

if AZURE_OPENAI_ENDPOINT and AZURE_OPENAI_API_KEY:
    client = AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
    )

    recovery_manager = TimeoutRecoveryManager(
        client=client,
        default_deployment=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
        max_retries=4,
    )

    # ── Test 1: Normal request (no timeout) ──
    print("\n🟢 Test 1: Normal request (no injection)")
    timeout_config = TimeoutConfig(
        ttft_timeout=10.0,
        read_timeout=10.0,
        total_timeout=30.0,
    )

    messages = [
        {"role": "user", "content": "Say 'hello' only."}
    ]

    result = recovery_manager.complete_with_recovery(
        messages=messages,
        timeout_config=timeout_config,
        max_tokens=20,
    )

    print(f"\n   ✅ Result:")
    print(f"      Success: {result.success}")
    print(f"      Strategy: {result.strategy_used.value}")
    print(f"      Attempts: {result.attempts}")
    print(f"      Response: {result.response}")

    # ── Test 2: Inject timeout → recovery succeeds ──
    print(f"\n{'─'*60}")
    print("🔥 Test 2: First 2 attempts timeout, 3rd succeeds")

    def real_create(**kw):
        return client.chat.completions.create(**kw)

    injector = RecoveryTimeoutInjector(
        real_create,
        fail_attempts=2,
        delay_seconds=5.0,  # Will exceed read_timeout
    )
    recovery_manager.timeout_injector = injector

    tight_timeout = TimeoutConfig(
        ttft_timeout=5.0,
        read_timeout=2.0,   # Tight — injected delay will exceed
        total_timeout=10.0,
    )

    messages = [
        {"role": "user", "content": "What is 2+2? Answer only."}
    ]

    result = recovery_manager.complete_with_recovery(
        messages=messages,
        timeout_config=tight_timeout,
        max_tokens=20,
        strategies=[
            RecoveryStrategy.RETRY_SAME,
            RecoveryStrategy.RETRY_SAME,
            RecoveryStrategy.RETRY_SAME,
        ],
    )

    print(f"\n   {'✅' if result.success else '❌'} Result:")
    print(f"      Success: {result.success}")
    print(f"      Strategy: {result.strategy_used.value}")
    print(f"      Attempts: {result.attempts}")
    print(f"      Timed out (any): {result.timed_out}")
    print(f"      Response: {result.response}")

    # Reset injector
    recovery_manager.timeout_injector = None

    # ── Test 3: All attempts timeout → partial response ──
    print(f"\n{'─'*60}")
    print("🔥 Test 3: All attempts timeout → return partial")

    always_fail_injector = RecoveryTimeoutInjector(
        real_create,
        fail_attempts=99,  # Always fail
        delay_seconds=5.0,
    )
    recovery_manager.timeout_injector = always_fail_injector

    result = recovery_manager.complete_with_recovery(
        messages=[
            {"role": "user", "content": "Count 1 2 3 4 5"}
        ],
        timeout_config=tight_timeout,
        max_tokens=50,
        strategies=[
            RecoveryStrategy.RETRY_SAME,
            RecoveryStrategy.RETRY_SIMPLIFIED,
            RecoveryStrategy.PARTIAL_RESPONSE,
        ],
    )

    print(f"\n   {'✅' if result.success else '❌'} Result:")
    print(f"      Success: {result.success}")
    print(f"      Strategy: {result.strategy_used.value}")
    print(f"      Attempts: {result.attempts}")
    print(f"      Partial: {result.partial}")
    print(f"      Response: {result.response or '(none)'}")

    # Reset
    recovery_manager.timeout_injector = None

else:
    print(
        "⚠️ AZURE_OPENAI_ENDPOINT or "
        "AZURE_OPENAI_API_KEY not set"
    )

📊 Timeout Recovery Demo

🟢 Test 1: Normal request (no injection)
   🔄 Attempt 1: retry_same (max_tokens=20)



   ✅ Result:
      Success: True
      Strategy: retry_same
      Attempts: 1
      Response: hello

────────────────────────────────────────────────────────────
🔥 Test 2: First 2 attempts timeout, 3rd succeeds
   🔄 Attempt 1: retry_same (max_tokens=20)
      💤 Injecting 5.0s delay (attempt 1)...
      ⚠️ read_timeout exceeded (5.2s)
   🔄 Attempt 2: retry_same (max_tokens=20)
      💤 Injecting 5.0s delay (attempt 2)...
      ⚠️ read_timeout exceeded (5.2s)
   🔄 Attempt 3: retry_same (max_tokens=20)

   ✅ Result:
      Success: True
      Strategy: retry_same
      Attempts: 3
      Timed out (any): False
      Response: 4

────────────────────────────────────────────────────────────
🔥 Test 3: All attempts timeout → return partial
   🔄 Attempt 1: retry_same (max_tokens=50)
      💤 Injecting 5.0s delay (attempt 1)...
      ⚠️ read_timeout exceeded (5.2s)
      📉 Reducing max_tokens: 50 → 25
   🔄 Attempt 2: retry_simplified (max_tokens=25)
      💤 Injecting 5.0s delay (attempt 2)...
    

## Part 5: Production Timeout Manager

Complete timeout management solution.

In [7]:
# Production Timeout Manager

@dataclass
class TimeoutResult:
    """Result with full timeout information."""
    success: bool
    response: Optional[str]
    timings: TimingResult
    recovery_used: bool
    recovery_strategy: Optional[RecoveryStrategy]
    total_attempts: int

class ProductionTimeoutManager:
    """
    Production-ready timeout management with:
    - Adaptive timeout calculation
    - TTFT monitoring
    - Streaming timeout handling
    - Automatic recovery
    """
    
    def __init__(
        self,
        client: AzureOpenAI,
        default_deployment: str,
        enable_adaptive_timeouts: bool = True,
        enable_recovery: bool = True,
    ):
        self.client = client
        self.default_deployment = default_deployment
        self.enable_adaptive = enable_adaptive_timeouts
        self.enable_recovery = enable_recovery
        
        self.calculator = AdaptiveTimeoutCalculator()
        self.recovery_manager = TimeoutRecoveryManager(
            client=client,
            default_deployment=default_deployment,
        )
        
        self.stats = {
            "total_requests": 0,
            "successful": 0,
            "timed_out": 0,
            "recovered": 0,
            "avg_ttft": 0.0,
            "ttft_samples": [],
        }
    
    def _estimate_tokens(self, messages: List[Dict[str, str]]) -> int:
        """Rough estimate of input tokens."""
        total_chars = sum(len(m.get("content", "")) for m in messages)
        return total_chars // 4  # Rough approximation
    
    def complete(
        self,
        messages: List[Dict[str, str]],
        max_tokens: int = 1000,
        timeout_config: Optional[TimeoutConfig] = None,
        is_complex: bool = False,
        has_tools: bool = False,
        **kwargs
    ) -> TimeoutResult:
        """
        Make a completion with full timeout management.
        """
        self.stats["total_requests"] += 1
        
        # Calculate adaptive timeouts if enabled
        if timeout_config is None and self.enable_adaptive:
            characteristics = RequestCharacteristics(
                estimated_input_tokens=self._estimate_tokens(messages),
                max_output_tokens=max_tokens,
                model=self.default_deployment,
                is_complex_task=is_complex,
                has_tools=has_tools,
            )
            timeout_config = self.calculator.calculate_timeouts(characteristics)
        elif timeout_config is None:
            timeout_config = TimeoutConfig()
        
        # Measure timing
        timing = measure_ttft_streaming(
            client=self.client,
            messages=messages,
            deployment=self.default_deployment,
            config=timeout_config,
            max_tokens=max_tokens,
            **kwargs
        )
        
        # Update TTFT stats
        if timing.ttft:
            self.stats["ttft_samples"].append(timing.ttft)
            self.stats["avg_ttft"] = sum(self.stats["ttft_samples"]) / len(self.stats["ttft_samples"])
        
        # Check if recovery needed
        recovery_used = False
        recovery_strategy = None
        total_attempts = 1
        
        if timing.timed_out and self.enable_recovery:
            self.stats["timed_out"] += 1
            print(f"   ⚠️ Timeout occurred ({timing.timeout_phase}), attempting recovery...")
            
            recovery_result = self.recovery_manager.complete_with_recovery(
                messages=messages,
                timeout_config=timeout_config,
                max_tokens=max_tokens,
            )
            
            recovery_used = True
            recovery_strategy = recovery_result.strategy_used
            total_attempts += recovery_result.attempts
            
            if recovery_result.success:
                self.stats["recovered"] += 1
                timing.response = recovery_result.response
                timing.timed_out = False
        
        if not timing.timed_out:
            self.stats["successful"] += 1
        
        return TimeoutResult(
            success=not timing.timed_out,
            response=timing.response,
            timings=timing,
            recovery_used=recovery_used,
            recovery_strategy=recovery_strategy,
            total_attempts=total_attempts,
        )
    
    def get_stats(self) -> Dict[str, Any]:
        """Get timeout statistics."""
        stats = self.stats.copy()
        del stats["ttft_samples"]  # Don't include raw samples
        
        if self.stats["total_requests"] > 0:
            stats["success_rate"] = self.stats["successful"] / self.stats["total_requests"] * 100
            stats["timeout_rate"] = self.stats["timed_out"] / self.stats["total_requests"] * 100
            if self.stats["timed_out"] > 0:
                stats["recovery_rate"] = self.stats["recovered"] / self.stats["timed_out"] * 100
            else:
                stats["recovery_rate"] = 0
        
        return stats

# Demonstrate production timeout manager
print("📊 Production Timeout Manager Demo")
print("=" * 60)

if AZURE_OPENAI_ENDPOINT and AZURE_OPENAI_API_KEY:
    client = AzureOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
    )
    
    manager = ProductionTimeoutManager(
        client=client,
        default_deployment=AZURE_OPENAI_CHAT_DEPLOYMENT_NAME,
        enable_adaptive_timeouts=True,
        enable_recovery=True,
    )
    
    # Make several requests
    print("\n📝 Making requests with timeout management...")
    
    queries = [
        ("What is 2+2?", False),
        ("Explain machine learning in 2 sentences.", False),
        ("Write a short poem about coding.", True),
    ]
    
    for query, is_complex in queries:
        print(f"\n   Query: {query[:40]}...")
        result = manager.complete(
            messages=[{"role": "user", "content": query}],
            max_tokens=100,
            is_complex=is_complex,
        )
        
        status = "✅" if result.success else "❌"
        print(f"   {status} TTFT: {result.timings.ttft*1000:.0f}ms, Total: {result.timings.total_time*1000:.0f}ms")
        if result.recovery_used:
            print(f"   ⚡ Recovery: {result.recovery_strategy.value}")
    
    # Display stats
    stats = manager.get_stats()
    print(f"\n📈 Timeout Manager Statistics:")
    print(f"   Total requests: {stats['total_requests']}")
    print(f"   Success rate: {stats.get('success_rate', 0):.1f}%")
    print(f"   Timeout rate: {stats.get('timeout_rate', 0):.1f}%")
    print(f"   Recovery rate: {stats.get('recovery_rate', 0):.1f}%")
    print(f"   Avg TTFT: {stats['avg_ttft']*1000:.0f}ms")
else:
    print("⚠️ AZURE_OPENAI_ENDPOINT or AZURE_OPENAI_API_KEY not set")

📊 Production Timeout Manager Demo

📝 Making requests with timeout management...

   Query: What is 2+2?...
   ✅ TTFT: 670ms, Total: 673ms

   Query: Explain machine learning in 2 sentences....
   ✅ TTFT: 1481ms, Total: 1486ms

   Query: Write a short poem about coding....
   ✅ TTFT: 1775ms, Total: 1784ms

📈 Timeout Manager Statistics:
   Total requests: 3
   Success rate: 100.0%
   Timeout rate: 0.0%
   Recovery rate: 0.0%
   Avg TTFT: 1309ms


## Best Practices Summary

### Timeout Configuration Guidelines

| Scenario | Connect | TTFT | Read | Total |
|----------|---------|------|------|-------|
| Simple queries | 5s | 10s | 30s | 60s |
| Complex analysis | 5s | 30s | 60s | 180s |
| Code generation | 5s | 30s | 60s | 300s |
| Reasoning (o1/o3) | 10s | 60s | 120s | 600s |

### Recovery Strategy Priority

```
┌─────────────────────────────────────────────────────────────────┐
│                   Timeout Recovery Flow                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   Timeout     1. Retry Same    2. Simplified    3. Faster      │
│   Occurs  →   Parameters   →   (½ tokens)   →   Model      →   │
│                                                                 │
│          4. Return Partial    5. Fail Fast                     │
│      →   (if available)   →   (no response)                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

## Wrap-up

### What You Learned

1. **TTFT vs Total**: Different timeouts for different stages
2. **Streaming Timeouts**: Monitor chunk-by-chunk progress
3. **Adaptive Timeouts**: Adjust based on request complexity
4. **Recovery Strategies**: Handle timeouts gracefully
5. **Production Manager**: Combine all strategies

### Additional Resources

- [Azure OpenAI Latency](https://learn.microsoft.com/en-us/azure/ai-services/openai/quotas-limits)
- [Timeout Best Practices](https://learn.microsoft.com/en-us/azure/architecture/best-practices/retry-service-specific)
- [Streaming Responses](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/streaming)

### Next Steps

- **4_observability/1_tracing_and_logging.ipynb**: Monitor and debug your applications